In [ ]:
pip install firebase-admin
pip install webdriver-manager

In [ ]:
import firebase_admin
from firebase_admin import credentials, firestore
import pandas as pd

# 서비스 계정 키 JSON 파일 경로 설정
cred = credentials.Certificate("path/almond-auth-firebase-adminsdk-2t243-049b004759.json")
firebase_admin.initialize_app(cred)

# Firestore 클라이언트 생성
db = firestore.client()

# Firebase Firestore에서 데이터 가져오기
def fetch_products_from_firestore():
    products_ref = db.collection('products')  # 'products'는 Firestore의 컬렉션 이름
    docs = products_ref.stream()
    
    # Firestore 데이터를 딕셔너리로 변환하여 DataFrame 생성
    products = []
    for doc in docs:
        product = doc.to_dict()
        product['id'] = doc.id  # Firestore 문서 ID를 포함하도록 설정
        products.append(product)
    
    return pd.DataFrame(products)

# DataFrame 생성
df = fetch_products_from_firestore()

# DataFrame 출력
print(df)


In [ ]:
productName = df['productName']
productName


In [3]:
productName_top_10 = df['productName'].head(10)

In [31]:
productName_top_20 = df['productName'].head(20)

In [6]:
import requests
from bs4 import BeautifulSoup
 
import pandas as pd
from datetime import datetime
import time

import re
 
# selenium import
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from datetime import datetime, timedelta
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

In [27]:
#드라이버 세팅
def setup_driver():
    options = webdriver.ChromeOptions()
    options.add_argument("window-size=1920x1080")
    options.add_argument("disable-gpu")
    options.add_argument("disable-infobars")
    options.add_argument("--disable-extensions")
    options.add_argument('--no-sandbox')
    
    chrome_driver_path = 'C:/chromedriver_win32/chromedriver.exe'
    service = Service(executable_path=chrome_driver_path)
    driver = webdriver.Chrome(service=service, options=options)
    return driver

In [28]:
#상품 존재 확인
def check_product_existence(driver):
    try:
        if driver.find_element(By.CLASS_NAME, 'NPaID1GO8x'):
            print("상품없음")
            return False
    except Exception:
        pass
    print("상품있음")
    return True

In [29]:
#상품 상세페이지 링크 추출
def extract_product_link(driver):
    try:
        ul_tag = driver.find_element(By.CLASS_NAME, 'wOWfwtMC_3._3cLKMqI7mI')
        first_li_tag = ul_tag.find_element(By.TAG_NAME, 'li')
        a_tag = first_li_tag.find_element(By.TAG_NAME, 'a')
        return a_tag.get_attribute('href')
    except Exception as e:
        print(f"상품링크 추출 실패 : {e}")
        return None

In [ ]:
driver = setup_driver()
# wait seconds...
driver.implicitly_wait(3)

write_dt_lst = []
item_nm_lst = []
content_lst = []

# 각 제품 이름에 대해 검색 및 리뷰 크롤링
for product_name in productName_top_20:
    # 검색 URL 구성
    search_url = f"https://smartstore.naver.com/adafi/search?q={product_name}"
    
    # 검색 페이지 요청
    driver.get(search_url)
    time.sleep(3)  # 페이지 로딩 대기
    
    try:
        productExist = check_product_existence(driver)
        
        product_link = None
        if productExist:
            product_link = extract_product_link(driver)
        else:
            continue
        
        if not product_link:
            print(f"Failed to extract product link for {product_name}")
            continue
        
        # 제품 페이지로 이동
        driver.get(product_link)
        print('product_link : ', product_link)

        driver.find_element(By.CSS_SELECTOR,'#content > div > div.z7cS6-TO7X > div._27jmWaPaKy > ul > li:nth-child(2) > a').click()
        time.sleep(3)

        # 1. 셀레니움으로 html가져오기
        html_source = driver.page_source
        # 2. bs4로 html 파싱
        soup = BeautifulSoup(html_source, 'html.parser')
        time.sleep(0.5)

        # 3. 리뷰 정보 가져오기
        reviews = soup.findAll('li', {'class': 'BnwL_cs1av'})



        # 4. 한페이지 내에서 수집 가능한 리뷰 리스트에 저장
        for review in range(len(reviews)):
            
            # 4-1.리뷰작성일자 수집
            write_dt_raw = reviews[review].findAll('span' ,{'class' : '_2L3vDiadT9'})[0].get_text()
            write_dt = datetime.strptime(write_dt_raw, '%y.%m.%d.').strftime('%Y%m%d')
            print('write_dt : ', write_dt)
            
            # 4-2.상품명 수집
            # 4-2-(1) 상품명이 포함된 css 선택자 입력 
            item_nm_info_raw = reviews[review].findAll('div', {'class' : '_2FXNMst_ak'})[0].get_text()
            print('item_nm_info_raw : ', item_nm_info_raw)

            # 4-2-(2) re.sub() 를 활용해 dl class="XbGQRlzveO"부분부터 추출한 문장을 공백으로 대체
            item_nm_info_for_del = reviews[review].findAll('div', {'class' : '_2FXNMst_ak'})[0].find('dl', {'class' : 'XbGQRlzveO'}).get_text()

            # 4-2-(3) re.sub(pattern, replacement, string) : string에서 pattern에 해당하는 부분을 replacement로 모두 대체
            item_nm_info= re.sub(item_nm_info_for_del, '', item_nm_info_raw)

            # 4-2-(4) find() : 문자열 순서 (인덱스) 반환 : find()를 활용해 '제품 선택 : '이 나오는 인덱스 반환
            str_start_idx = re.sub(item_nm_info_for_del, '', item_nm_info_raw).find('제품 선택: ')

            # 4-2-(5) 제품명만 추출. strip(): 공백 제거 
            item_nm = item_nm_info[str_start_idx + 6:].strip()
            

            # 4-3. 리뷰내용 수집
            review_content_raw = reviews[review].findAll('div', {'class' : '_1kMfD5ErZ6'})[0].find('span', {'class' : '_2L3vDiadT9'}).get_text()
            review_content = re.sub(' +', ' ',re.sub('\n',' ',review_content_raw ))
            print('review_content : ', review_content)
            
            # 4-4. 수집데이터 저장
            write_dt_lst.append(write_dt)
            item_nm_lst.append(item_nm)
            content_lst.append(review_content)
        
    except Exception as e:
        print(f"Error processing {product_name}: {e}")

# 수집한 데이터를 DataFrame으로 변환
reviews_df = pd.DataFrame({
    'Write Date': write_dt_lst,
    'Item Name': item_nm_lst,
    'Review Content': content_lst
})

# DataFrame 출력
print(reviews_df)

In [ ]:
reviews_df